In [75]:
import ast 
import importlib
import re

import pandas as pd

Partiamo caricando i tuoi dati d'esempio

In [45]:
df = pd.read_csv('test.csv')
df

,Nome,Cognome,Età,Città,Professione,Sesso,Salario
0,Mario,Rossi,35,Roma,Ingegnere,M,50000
1,Laura,Bianchi,28,Milano,Avvocato,F,60000
2,Giovanni,Verdi,42,Napoli,Medico,M,80000
3,Francesca,Russo,31,Torino,Insegnante,F,45000
4,Paolo,Conti,40,Roma,Imprenditore,M,100000
5,Giulia,Ferrari,27,Milano,Architetto,F,55000
6,Andrea,Bianchi,33,Napoli,Ingegnere,M,70000
7,Martina,Rossi,29,Torino,Avvocato,F,60000
8,Luca,Verdi,39,Roma,Medico,M,85000


Per prima cosa, ti suggerirei di usare [pandas.Dataframe.eval](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.eval.html#pandas.DataFrame.eval) e non [pandas.eval](https://pandas.pydata.org/docs/reference/api/pandas.eval.html#pandas.eval) perché si occupa già da solo di "popolare" il namespace con le variabili derivanti dalle colonne del dataframe.

In [46]:
df.eval('Età > 40')

0    False
1    False
2     True
3    False
4    False
5    False
6    False
7    False
8    False
Name: Età, dtype: bool

A questo punto l'unica cosa che ti serve è "iniettare" le funzioni definite dall'utente. Per prima cosa, vediamo come estrarre in modo programmatico le funzioni da un modulo dato il suo nome (in modo tale che l'utente possa specificare il nome del modulo da UI).

In [47]:
module_name = 'functions_file'
module = importlib.import_module(module_name)

Se conosci il nome della funzione, è facile trovarla

In [48]:
function_name = 'raddoppia'
f = getattr(module, function_name)

In [49]:
f(4)

8

Ora il punto potrebbe essere come determinare i nomi di funzione a partire dall'espressione introdotta dall'utente. 

Eviterei tentativi basati sulle espressioni regolari, dato che il linguaggio delle espressioni non è regolare! 

Per convincertene, guarda questo esempio che "complica" il tuo avvolgendo in un `raddoppia` entrambe i mebbri della tua prima disequazione:

In [74]:
espressione = "(raddoppia(raddoppia(Età)) > raddoppia(60)) & ((aggiungi10(Salario) > 5000) | (Sesso == 'F'))"

Questo introducen una chiamata di chiamata, che con la tua espressione regolare
non potresti gestire

In [78]:
def parse_function_call(condizione):
  pattern = r"(\w+)\((\w+)\)"
  matches = re.findall(pattern, condizione)

  for match in matches:
    function_name, column_name = match
    return function_name, column_name

In [80]:
parse_function_call(espressione)

('raddoppia', 'Età')


Per fortuna puoi usare il parser di Python stesso.

In [81]:
pt = ast.parse(espressione)

Come puoi notare, il *parse tree* è un albero i cui nodi sono istanze di classi che derivano da [ast.AST](https://docs.python.org/3/library/ast.html#ast.AST). In particolare, le invocazioni di funzione sono rappresentate da nodi di tipo [ast.Call](https://docs.python.org/3/library/ast.html#ast.Call).

In [82]:
ast.dump(pt)

"Module(body=[Expr(value=BinOp(left=Compare(left=Call(func=Name(id='raddoppia', ctx=Load()), args=[Call(func=Name(id='raddoppia', ctx=Load()), args=[Name(id='Età', ctx=Load())], keywords=[])], keywords=[]), ops=[Gt()], comparators=[Call(func=Name(id='raddoppia', ctx=Load()), args=[Constant(value=60)], keywords=[])]), op=BitAnd(), right=BinOp(left=Compare(left=Call(func=Name(id='aggiungi10', ctx=Load()), args=[Name(id='Salario', ctx=Load())], keywords=[]), ops=[Gt()], comparators=[Constant(value=5000)]), op=BitOr(), right=Compare(left=Name(id='Sesso', ctx=Load()), ops=[Eq()], comparators=[Constant(value='F')]))))], type_ignores=[])"

Puoi andare a caccia di tali nodi con una visita ricorsiva del parse tree

In [83]:
def find_functions(node):
  res = set()
  for child in ast.iter_child_nodes(node):
    if isinstance(child, ast.Call):
      res.add(child.func.id)
    res |= find_functions(child)
  return res

In [84]:
find_functions(pt)

{'aggiungi10', 'raddoppia'}

Questo ti basta per preparare un dizionario che associ ad ogni nome di funzione la funzione caricata dal modulo.

In [85]:
name2func = {name: getattr(module, name) for name in find_functions(pt)}

In [86]:
name2func

{'aggiungi10': <function functions_file.aggiungi10(y)>,
 'raddoppia': <function functions_file.raddoppia(x)>}

Sei pronto per valutare l'espressione

In [87]:
df.eval(espressione, resolvers = [name2func], engine = 'python')

0     True
1    False
2     True
3     True
4     True
5    False
6     True
7    False
8     True
dtype: bool